In [80]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import convolve

class Robot:
    def __init__(self, track_len, kernel=[1.0], sensor_accuracy=0.9):
        self.track_len = track_len
        self.pos = 0
        self.kernel = kernel
        self.sensor_accuracy = sensor_accuracy

    def move(self, distance=1):
        self.pos += distance
        r = np.random.uniform(low=0,high=1)
        s = 0
        offset = -(len(self.kernel) - 1) / 2
        for k in self.kernel:
            s += k
            if r <= s:
                break
            offset += 1
        self.pos = int((self.pos + offset) % self.track_len)
        return self.pos

    def sense_gaussian(self):
        return int((self.pos + np.random.normal(0,1)) % self.track_len)
    
    def sense(self):
        pos = self.pos
        if np.random.uniform(low=0,high=1) > self.sensor_accuracy:
            pos += np.random.choice([-1,1])
            pos = pos % self.track_len
        return pos

def robot_filter(iterations, kernel, sensor_accuracy, move_distance):
    track = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    prior = np.array([0.9] + [0.01]*9)
    posterior = prior[:]
    normalize(prior)
    robot = Robot(len(track), kernel, sensor_accuracy)
    for i in range(iterations):
        # move the robot
        robot.move(distance=move_distance)

        # predict
        prior = predict(posterior, move_distance, kernel)

        # update our filter
        measurement = robot.sense()
        likelihood = compute_sensor_likelihood(track, measurement, sensor_accuracy)
        posterior = update(likelihood, prior)
    return posterior,robot.pos

def compute_sensor_likelihood(hall, z, z_prob):
    if z_prob == 1.0:
        scale = 1e8  # Handle the case of perfect sensor accuracy
    else:
        scale = z_prob / (1.0 - z_prob)
    likelihood = np.ones(len(hall))
    likelihood[hall == z] *= scale
    return likelihood

def normalize(pdf):
    return pdf / np.sum(pdf)

def predict(pdf, offset, kernel):
    return convolve(np.roll(pdf, offset), kernel)

def update(likelihood, prior):
    posterior = prior * likelihood
    return normalize(posterior)

def test_robot_filter():
    kernel = [0.1, 0.6, 0.3]
    estimate,real_pos = robot_filter(1000, kernel=kernel, sensor_accuracy=0.5, move_distance=4)
    est_pos = np.argmax(estimate)
    return est_pos == real_pos

def main():
    total = []
    for _ in range(1000):
        total.append(test_robot_filter())
    print(np.sum(total) / 1000)

main()


0.097
